In [7]:
import os 
import pandas as pd
import numpy as np
import os
from pathlib import Path

relative_path = r'C:\Users\Hsieh\Documents\nccucs\specialTopic\special_topic\src\data_process\transition__chord_matrix'
def get_chord_file(relative_path) -> list:
    """
    get the chord file path list

    :param relative: the relative path
    :return: chord_file_list
    """

    chord_file_list = []

    # get the file in the upper folder POP909
    parent_path = Path(relative_path).parent
    file_list = os.listdir(os.path.join(parent_path, 'POP909'))

    # remove file that fileName is not number
    for file in file_list:
        if not file.isdigit():
            file_list.remove(file)

    for file in file_list:

        chord_file = os.path.join(parent_path, 'POP909', file, 'chord_midi.txt')
        chord_file_list.append(chord_file)

    return chord_file_list


get_chord_file(relative_path)

['C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\001\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\002\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\003\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\004\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\005\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\006\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\007\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_topic\\src\\data_process\\POP909\\008\\chord_midi.txt',
 'C:\\Users\\Hsieh\\Documents\\nccucs\\specialTopic\\special_top

In [8]:
def data_preprocess(chord_df) -> pd.DataFrame:
    """
    data preprocess to remove empty chord and add start_chord and end_chord
    :param chord_df: the chord DataFrame
    :return: chord_df
    """

    # remove the empty chord
    chord_df = chord_df[chord_df['chord'] != 'N']

    start_chord = pd.Series(
        {'start_time': 0, 'end_time': chord_df['start_time'].iloc[0], 'chord': 'start_chord'})
    end_chord = pd.Series({'start_time': chord_df['end_time'].iloc[-1],
                          'end_time': chord_df['end_time'].iloc[-1]+1, 'chord': 'end_chord'})

    chord_df = pd.concat([pd.DataFrame(start_chord).T,
                         chord_df], ignore_index=True)

    chord_df = pd.concat(
        [chord_df, pd.DataFrame(end_chord).T], ignore_index=True)

    # simplfy the chord by change remove chord after '/' and remove chord after '(')
    chord_df['chord'] = chord_df['chord'].apply(lambda x: x.split('/')[0])
    chord_df['chord'] = chord_df['chord'].apply(lambda x: x.split('(')[0])
    return chord_df


def merge_all_df(file_list) -> pd.DataFrame:
    """
    merge all chord_df
    :param file_list: the chord file list
    :return: all_df
    """

    all_df = pd.DataFrame()

    for index, file in enumerate(file_list):

        chord_df = pd.read_csv(file, sep='\t', header=None, names=[
                               'start_time', 'end_time', 'chord'])

        chord_df = data_preprocess(chord_df)
        chord_df['song_num'] = index + 1

        all_df = pd.concat([all_df, chord_df])


  
    all_df.to_csv(r"csv_file/all_chord.csv", index=True, header=True)
    return all_df

merge_all_df(get_chord_file(relative_path))


,start_time,end_time,chord,song_num
0,0,2.721993,start_chord,1
1,2.721993,4.055323,B:maj,1
2,4.055323,5.388653,C#:maj,1
3,5.388653,6.721983,Bb:min,1
4,6.721983,8.055313,Eb:min,1
...,...,...,...,...
48,91.484151,93.791843,C:maj,909
49,93.791843,96.099535,G:maj,909
50,96.099535,97.253381,C:maj,909
51,97.253381,98.98415,C:maj,909
